In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn as sk
import matplotlib.pyplot as plt

In [3]:
with open('lab.txt', 'r') as f:
    labels_org = f.read()
df=pd.read_csv('Seq_Embo_f.csv',header=None)
features=np.array(df[:][:]).astype(np.float32)
#labels = np.array([int(_)  for _ in labels_org.split()]).astype(np.int64)
labels =np.array([int(_)  for _ in labels_org.split()]).astype(np.float32)
train_x,test_x,train_y,test_y=train_test_split(features,labels,test_size=0.2)
#train_x=train_x.reshape(-1,30,30)
#test_x=test_x.reshape(-1,30,30)
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), "\nTest set: \t\t{}".format(test_x.shape))
print("label set: \t\t{}".format(train_y.shape), "\nTest label set: \t\t{}".format(test_y.shape))

def get_batches(x, y, batch_size=800):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

			Feature Shapes:
Train set: 		(800, 900) 
Test set: 		(200, 900)
label set: 		(800,) 
Test label set: 		(200,)


In [4]:
seq_len=30
embeding_size=30
n_input = embeding_size*seq_len
n_classes = 1
learning_rate = 0.01
training_iters = 200000
batch_size = 800
display_step = 10

In [10]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None])
keep_prob = tf.placeholder(tf.float32) 
dropout=1
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

def conv_net(x, weights, biases, dropout):
    global embeding_size
    global seq_len
    x = tf.reshape(x, shape=[-1, embeding_size, seq_len, 1])
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, keep_prob)
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 6 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 6])),
    # 5x5 conv, 6 inputs, 12 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 6, 12])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([30*30*12, 150])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([150, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([6])),
    'bc2': tf.Variable(tf.random_normal([12])),
    'bd1': tf.Variable(tf.random_normal([150])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [15]:

pred = conv_net(x, weights, biases, keep_prob)
pred=tf.nn.tanh(pred)
pp=tf.reshape(pred,[-1])
cost=tf.nn.l2_loss(y-pred)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(cost)
total_error = tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
unexplained_error = tf.reduce_sum(tf.square(y - pp))
accuracy = 1- tf.div(total_error, unexplained_error)
init = tf.global_variables_initializer()

In [16]:
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    print('Initialized')
    for epoch in range(10):
        for io,(xx,yy) in enumerate(get_batches(train_x, train_y, 800), 1):
            X_batch  = xx
            y_batch  = yy
            _,acc_train=sess.run([training_op, accuracy],feed_dict={x: X_batch, y: y_batch,keep_prob: 0.1})
        acc_test = accuracy.eval(feed_dict={x: test_x, y: test_y, keep_prob: 1})
        pred_test=pred.eval(feed_dict={x: test_x, y: test_y, keep_prob: 1})
        print(epoch, "Train accuracy:",acc_train , "Test accuracy:", acc_test)
        saver.save(sess, "checkpoints/sento-anal_f1_CNN_enbedno_2.ckpt")

Initialized
0 Train accuracy: 0.740497 Test accuracy: 0.778982
1 Train accuracy: 0.72356 Test accuracy: 0.778982
2 Train accuracy: 0.724796 Test accuracy: 0.778982
3 Train accuracy: 0.737726 Test accuracy: 0.778982
4 Train accuracy: 0.737167 Test accuracy: 0.778982
5 Train accuracy: 0.737165 Test accuracy: 0.778982
6 Train accuracy: 0.749891 Test accuracy: 0.778982
7 Train accuracy: 0.743206 Test accuracy: 0.778982
8 Train accuracy: 0.737165 Test accuracy: 0.778982
9 Train accuracy: 0.738288 Test accuracy: 0.778982


In [ ]:
plt.plot()